In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

%matplotlib inline

In [6]:
def readAndShowImageAndFilter(image,filtr):
    im = util.img_as_float(io.imread(image, as_gray=True))
    im_filtr = util.img_as_float(io.imread(filtr, as_gray=True))
    io.imshow_collection([im,im_filtr], cmap='gray')
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Wireboard':'/home/jim-alexander/Dokumenter/bildebehandling/Bilder/image_resources/crabpulsar-optical.tif'
    },
    description='Bilde: ',
    disabled=False,
)

inputFilter = widgets.Dropdown(
    options={
        'Cross':'se_cross.pbm',
        'Box':'boxSE.png'
    },
    description='Filter: ',
    disabled=False,
)

interactive(readAndShowImageAndFilter, image=inputImage, filtr=inputFilter)

interactive(children=(Dropdown(description='Bilde: ', options={'Wireboard': '/home/jim-alexander/Dokumenter/bi…

In [31]:
def filter(image,se,func):
    se_height,se_width = se.shape

    pad_v = se_height//2
    pad_h = se_width//2

    padded = util.pad(image,(pad_v,pad_h),mode="constant")

    out = numpy.zeros(image.shape)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            subimage = padded[i:i+se_height,j:j+se_width]
            s_w, s_h = subimage.shape

            found = False
            cur = 0
            for x in range(s_w):
                for y in range(s_h):
                    if(se[x,y] != 0):
                        if(found != True):
                            cur = subimage[x,y]
                            found = True
                        else:
                            cur = func(cur,subimage[x,y])
                
            out[i,j] = cur
    return out
    
    

def dilation(im,se):
    return filter(im,se,max)

def erode(im,se):
    return filter(im,se,min)

def opening(image, se):
    return dilation(erode(image,se),se)

def closing(image, se):
    return erode(dilation(image,se),se)

In [32]:
inputMethod = widgets.Dropdown(
    options={'erode':erode,
             'dilation':dilation,
             'opening':opening,
             'closing':closing
            },
    description='Filter: ',
    disabled=False,
)

def update(image, filtr, method):
    im = io.imread(image, as_gray=True)
    se = io.imread(filtr, as_gray=True)
    
    im_prosessed = method(im,se)
    
    io.imshow_collection([im,im_prosessed], cmap='gray')
    io.show()
    
interactive(update, image=inputImage, filtr=inputFilter, method=inputMethod)

interactive(children=(Dropdown(description='Bilde: ', options={'Wireboard': '/home/jim-alexander/Dokumenter/bi…